# Limpieza de Datos Dataset Promociones-20190715

In [1]:
import pandas as pd
from utils_functions import convert_file, upload_to_bucket

#### Importar el dataset y observar los primeros registros con head y verificar los tipos de datos con el método info.

In [2]:
promo_col_names = [
        'idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'ventas_estimadas_und',
        'nro_prom', 'fecha_trans', 'porcentaje_dcto'
]

In [3]:
promo_df = pd.read_csv("../data/VMI_Promociones-20190715.txt",
                       encoding="utf-8",
                       header=None,
                       delimiter='\t',
                       names=promo_col_names)

In [4]:
promo_df.head()

,idb,id_producto,fecha_desde,fecha_hasta,ventas_estimadas_und,nro_prom,fecha_trans,porcentaje_dcto
0,242,228217,20161023,20161030,0.98,436830,20161005,7.0
1,242,228217,20161023,20161030,0.98,436830,20161006,7.0
2,242,228217,20161023,20161030,0.98,436830,20161007,7.0
3,242,228217,20161023,20161030,0.98,436830,20161008,7.0
4,242,228217,20161023,20161030,0.98,436830,20161009,7.0


In [5]:
promo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285253 entries, 0 to 285252
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   idb                   285253 non-null  int64  
 1   id_producto           285253 non-null  int64  
 2   fecha_desde           285253 non-null  int64  
 3   fecha_hasta           285253 non-null  int64  
 4   ventas_estimadas_und  285253 non-null  float64
 5   nro_prom              285253 non-null  int64  
 6   fecha_trans           285253 non-null  int64  
 7   porcentaje_dcto       285253 non-null  float64
dtypes: float64(2), int64(6)
memory usage: 17.4 MB


#### Ajustes al dataframe
- Convertir la columnas fecha_desde, fecha_hasta y fecha_trans a tipo de dato datetime.
- Verificar si existen valores nulos y/o repetidos.
- Obtener las fechas máximas y minimas de transaciones.
- Crear una nueva columnas llamada duracion_promo para obtener el delta de tiempo entre la fecha_desde y la fecha_hasta de una promoción.
- Obtener una análisis descriptivo de los datos.

In [6]:
promo_df = convert_file(promo_df, 'fecha_desde',  'date')

In [7]:
promo_df = convert_file(promo_df, 'fecha_hasta',  'date')

In [10]:
promo_df = convert_file(promo_df, 'ventas_estimadas_und', 'int')

In [11]:
promo_df.isnull().sum(axis=0)

idb                     0
id_producto             0
fecha_desde             0
fecha_hasta             0
ventas_estimadas_und    0
nro_prom                0
fecha_trans             0
porcentaje_dcto         0
dtype: int64

In [12]:
# Identify the duplicate rows using a subset of columns
subset_cols = ['idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'nro_prom', 'ventas_estimadas_und']
duplicate_mask = promo_df.duplicated(subset=subset_cols, keep='first')

# Count the number of duplicate rows
total_duplicates = sum(duplicate_mask)

# Remove the duplicate rows from promo_df
promo_df = promo_df[~duplicate_mask]

print(f"El total de registros duplicados en el dataset es: {total_duplicates}")

El total de registros duplicados en el dataset es: 268692


Después de analizar los datos, hemos encontrado 268,692 registros duplicados en la tabla de promociones. Estos registros tienen la misma información, excepto por la fecha de transacción. Como esta información no es relevante en este caso, la hemos eliminado de nuestro conjunto de datos.

También hemos notado que la columna "porcentaje_dcto" parece tener información duplicada con la columna "duracion_promo". Después de revisar cuidadosamente los datos, hemos decidido eliminar la columna "porcentaje_dcto" y conservar solo la columna "duracion_promo" como la fuente de información para la duración de la promoción. La columna "duracion_promo" se ha creado calculando el tiempo transcurrido entre la fecha de inicio y la fecha de finalización de la promoción.

In [13]:
promo_df['duracion_promo'] = (promo_df.fecha_hasta - promo_df.fecha_desde).astype('timedelta64[D]').astype(int)

In [14]:
# group the dataframe by all columns except 'ventas_estimadas_und'
grouped = promo_df.groupby(['idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'nro_prom',  'duracion_promo'], as_index=False)

# sum the 'ventas_estimadas_und' column for each group
summed = grouped.agg({'ventas_estimadas_und': 'sum'})

# drop the duplicated rows
result = summed.drop_duplicates()

print(f"El total de registros después de eliminar duplicados es: {len(promo_df)}")

El total de registros después de eliminar duplicados es: 16561


In [15]:
min_date = result['fecha_desde'].min()
max_date = result['fecha_hasta'].max()


print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

Minimum date: 2016-07-19 00:00:00
Maximum date: 2019-07-16 00:00:00


In [16]:
result.describe()

,idb,id_producto,nro_prom,duracion_promo,ventas_estimadas_und
count,15270.000000,15270.000000,15270.000000,15270.000000,15270.000000
mean,7187.241257,388184.750491,472094.586182,8.320825,22.427898
std,3128.103999,84684.306020,38182.677713,2.885136,29.979450
min,242.000000,164146.000000,425196.000000,3.000000,0.000000
25%,5208.000000,342682.000000,436830.000000,7.000000,4.000000
50%,9036.000000,427445.000000,456807.000000,7.000000,12.000000
75%,9160.000000,458395.000000,502704.000000,8.000000,30.000000
max,9977.000000,507528.000000,541066.000000,18.000000,300.000000


#### Observaciones dataset Promociones-20190715

1. El conjunto de datos contiene información sobre promociones realizadas desde el 19 de julio de 2016 hasta el 16 de julio de 2019.

2. Hay un total de 15.270 registros y 8 columnas en el conjunto de datos, que se describen de la siguiente manera:

    idb: Identificador interno de Boca
    id_producto: Identificador interno del artículo
    fecha_desde: Fecha de inicio de la promoción
    fecha_hasta: Fecha de finalización de la promoción
    ventas_estimadas_und: Estimación de ventas en unidades durante la promoción
    nro_prom: Número de la promoción
    duracion_promo: Duración de la promoción (diferencia entre fecha_desde y fecha_hasta)

3. Observamos que hay 268.692 registros duplicados en la tabla de promociones. Estos registros comparten la misma fecha_desde,
fecha_hasta, id_producto, idb, nro_prom y ventas_estimadas_und, diferenciándose únicamente por el valor de fecha_transacción. Por lo tanto, podemos concluir con certeza que estos registros son duplicados.

4. Es importante tener en cuenta que esta tabla no registra transacciones, sino que registra el número de promoción, la fecha de inicio y final, y el valor estimado de las ventas en la promoción. Por esta razón, la fecha de transacción no es relevante en este caso.

5. La media de ventas estimadas en unidades durante las promociones es de 22 unidades.

6. La duración máxima de una promoción es de 18 días, mientras que la duración mínima es de 8 días, la media es de 8 días.

### Guardar el dataset limpio

In [17]:
promo_df.to_csv('../clean_data/promociones_clean.csv', index=False)

#### Subir csv a google cloud storage

In [18]:
upload_to_bucket('deodorants', '../clean_data/promociones_clean.csv', 'promociones')

File ../clean_data/promociones_clean.csv uploaded successfully to bucket deodorants/promociones
